In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp

from sklearn.model_selection import train_test_split

import os
import cv2
import shutil  
import numpy as np
import random
import datetime



In [2]:
%load_ext tensorboard

In [3]:
# memory fix
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [4]:
PATH = os.getcwd()
DATASET = os.path.join(PATH, "dataset")
CATEGORIES = [folder for folder in os.listdir(DATASET)]
INDEX_VALUES = [i for i in range(30)]

In [5]:
def create_training_data():
    _images = []
    _labels = []
    for category in CATEGORIES:
        class_num = CATEGORIES.index(category)
        new_path = os.path.join(DATASET, category)
        for img in os.listdir(new_path):
            img_array = cv2.imread(os.path.join(new_path, img), cv2.IMREAD_GRAYSCALE)
            _images.append(img_array)
            _labels.append(class_num)
    return (_images, _labels)

(training_images, training_labels) = create_training_data()
training_images = np.array(training_images)
training_images = training_images / 255.0

X_train, X_test, y_train, y_test = train_test_split(
    training_images, training_labels, test_size=0.3, random_state=42)

In [6]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32, 64, 128, 256]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.9))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['Adadelta', 'Adagrad', 'Adam', 'Adamax', 'FTRL', 'NAdam', 'RMSprop', 'SGD']))

METRIC_ACCURACY = 'accuracy'

In [7]:
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [8]:
def train_test_model(hparams): 
    log_dir = "logs\\fit\\" \
    + "neurons-" + str(hparams[HP_NUM_UNITS]) + " " \
    + "dropout-" + str(hparams[HP_DROPOUT]) + " " \
    + str(hparams[HP_OPTIMIZER]) + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(30, activation=tf.nn.softmax),
    ])

    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )

    model.fit(np.array(X_train), np.array(y_train), epochs=10) # Run with 1 epoch to speed things up for demo purposes
    _, accuracy = model.evaluate(np.array(X_train), np.array(y_train)
    ,
    callbacks=[
        tf.keras.callbacks.TensorBoard(log_dir),  # log metrics
        hp.KerasCallback(log_dir, hparams),  # log hparams
    ],)
    return accuracy

In [9]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [10]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
              HP_NUM_UNITS: num_units,
              HP_DROPOUT: dropout_rate,
              HP_OPTIMIZER: optimizer,
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'Adadelta'}
Train on 1050 samples
Epoch 1/10
1050/1050 [==============================] - 1s 589us/sample - loss: 3.4553 - accuracy: 0.0343
Epoch 2/10
1050/1050 [==============================] - 0s 64us/sample - loss: 3.4514 - accuracy: 0.0305
Epoch 3/10
1050/1050 [==============================] - 0s 62us/sample - loss: 3.4402 - accuracy: 0.0429
Epoch 4/10
1050/1050 [==============================] - 0s 66us/sample - loss: 3.4302 - accuracy: 0.0476
Epoch 5/10
1050/1050 [==============================] - 0s 70us/sample - loss: 3.4289 - accuracy: 0.0552
Epoch 6/10
1050/1050 [==============================] - 0s 66us/sample - loss: 3.4264 - accuracy: 0.0410
Epoch 7/10
1050/1050 [==============================] - 0s 65us/sample - loss: 3.4194 - accuracy: 0.0514
Epoch 8/10
1050/1050 [==============================] - 0s 65us/sample - loss: 3.4200 - accuracy: 0.0467
Epoch 9/10
1050/1050 [==============================

1050/1050 [==============================] - 0s 67us/sample - loss: 3.3626 - accuracy: 0.0562
Epoch 7/10
1050/1050 [==============================] - 0s 66us/sample - loss: 3.3376 - accuracy: 0.0571
Epoch 8/10
1050/1050 [==============================] - 0s 68us/sample - loss: 3.3273 - accuracy: 0.0752
Epoch 9/10
1050/1050 [==============================] - 0s 67us/sample - loss: 3.2786 - accuracy: 0.0752
Epoch 10/10
1050/1050 [==============================] - 0s 111us/sample - loss: 3.2107 - accuracy: 0.0714
--- Starting trial: run-7
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'SGD'}
Train on 1050 samples
Epoch 1/10
1050/1050 [==============================] - 0s 446us/sample - loss: 3.3920 - accuracy: 0.0352
Epoch 2/10
1050/1050 [==============================] - 0s 60us/sample - loss: 3.3412 - accuracy: 0.0648
Epoch 3/10
1050/1050 [==============================] - 0s 58us/sample - loss: 3.2947 - accuracy: 0.0800
Epoch 4/10
1050/1050 [==============================] - 0s 57us/sa

1050/1050 [==============================] - 0s 65us/sample - loss: 3.4009 - accuracy: 0.0390
Epoch 6/10
1050/1050 [==============================] - 0s 68us/sample - loss: 3.4011 - accuracy: 0.0343
Epoch 7/10
1050/1050 [==============================] - 0s 65us/sample - loss: 3.4007 - accuracy: 0.0295
Epoch 8/10
1050/1050 [==============================] - 0s 63us/sample - loss: 3.4007 - accuracy: 0.0295
Epoch 9/10
1050/1050 [==============================] - 0s 64us/sample - loss: 3.4007 - accuracy: 0.0352
Epoch 10/10
1050/1050 [==============================] - 0s 117us/sample - loss: 3.4006 - accuracy: 0.0381
--- Starting trial: run-13
{'num_units': 16, 'dropout': 0.9, 'optimizer': 'NAdam'}
Train on 1050 samples
Epoch 1/10
1050/1050 [==============================] - 0s 409us/sample - loss: 3.4812 - accuracy: 0.0200
Epoch 2/10
1050/1050 [==============================] - 0s 70us/sample - loss: 3.5190 - accuracy: 0.0229
Epoch 3/10
1050/1050 [==============================] - 0s 73us

1050/1050 [==============================] - 0s 110us/sample - loss: 2.4393 - accuracy: 0.2162
--- Starting trial: run-19
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'Adamax'}
Train on 1050 samples
Epoch 1/10
1050/1050 [==============================] - 0s 265us/sample - loss: 3.4101 - accuracy: 0.0438
Epoch 2/10
1050/1050 [==============================] - 0s 64us/sample - loss: 3.2933 - accuracy: 0.0629
Epoch 3/10
1050/1050 [==============================] - 0s 65us/sample - loss: 3.2233 - accuracy: 0.1000
Epoch 4/10
1050/1050 [==============================] - 0s 62us/sample - loss: 3.1612 - accuracy: 0.1067
Epoch 5/10
1050/1050 [==============================] - 0s 64us/sample - loss: 3.0963 - accuracy: 0.1267
Epoch 6/10
1050/1050 [==============================] - 0s 65us/sample - loss: 3.0529 - accuracy: 0.1390
Epoch 7/10
1050/1050 [==============================] - 0s 65us/sample - loss: 2.9644 - accuracy: 0.1657
Epoch 8/10
1050/1050 [==============================] - 0s 63us

1050/1050 [==============================] - 0s 64us/sample - loss: 3.3999 - accuracy: 0.0381
Epoch 6/10
1050/1050 [==============================] - 0s 62us/sample - loss: 3.4008 - accuracy: 0.0381
Epoch 7/10
1050/1050 [==============================] - 0s 63us/sample - loss: 3.4027 - accuracy: 0.0371
Epoch 8/10
1050/1050 [==============================] - 0s 63us/sample - loss: 3.4013 - accuracy: 0.0381
Epoch 9/10
1050/1050 [==============================] - 0s 64us/sample - loss: 3.4009 - accuracy: 0.0390
Epoch 10/10
1050/1050 [==============================] - 0s 108us/sample - loss: 3.3984 - accuracy: 0.0390
--- Starting trial: run-26
{'num_units': 32, 'dropout': 0.9, 'optimizer': 'Adam'}
Train on 1050 samples
Epoch 1/10
1050/1050 [==============================] - 0s 269us/sample - loss: 3.4899 - accuracy: 0.0333
Epoch 2/10
1050/1050 [==============================] - 0s 64us/sample - loss: 3.4020 - accuracy: 0.0352
Epoch 3/10
1050/1050 [==============================] - 0s 62us/

...

1050/1050 [==============================] - 0s 69us/sample - loss: 2.1766 - accuracy: 0.5286
Epoch 6/10
1050/1050 [==============================] - 0s 67us/sample - loss: 1.9497 - accuracy: 0.6324
Epoch 7/10
1050/1050 [==============================] - 0s 69us/sample - loss: 1.7234 - accuracy: 0.6733
Epoch 8/10
1050/1050 [==============================] - 0s 70us/sample - loss: 1.5796 - accuracy: 0.7210
Epoch 9/10
1050/1050 [==============================] - 0s 68us/sample - loss: 1.4074 - accuracy: 0.7705
Epoch 10/10
1050/1050 [==============================] - 0s 112us/sample - loss: 1.0933 - accuracy: 0.9019
--- Starting trial: run-52
{'num_units': 128, 'dropout': 0.1, 'optimizer': 'FTRL'}
Train on 1050 samples
Epoch 1/10
1050/1050 [==============================] - 0s 264us/sample - loss: 3.4102 - accuracy: 0.0286
Epoch 2/10
1050/1050 [==============================] - 0s 70us/sample - loss: 3.4012 - accuracy: 0.0286
Epoch 3/10
1050/1050 [==============================] - 0s 73us

1050/1050 [==============================] - 0s 118us/sample - loss: 3.3763 - accuracy: 0.0743
--- Starting trial: run-58
{'num_units': 128, 'dropout': 0.9, 'optimizer': 'Adam'}
Train on 1050 samples
Epoch 1/10
1050/1050 [==============================] - 0s 395us/sample - loss: 3.7169 - accuracy: 0.0219
Epoch 2/10
1050/1050 [==============================] - 0s 90us/sample - loss: 3.4035 - accuracy: 0.0371
Epoch 3/10
1050/1050 [==============================] - 0s 82us/sample - loss: 3.4078 - accuracy: 0.0371
Epoch 4/10
1050/1050 [==============================] - 0s 82us/sample - loss: 3.4016 - accuracy: 0.0343
Epoch 5/10
1050/1050 [==============================] - 0s 81us/sample - loss: 3.4005 - accuracy: 0.0362
Epoch 6/10
1050/1050 [==============================] - 0s 89us/sample - loss: 3.4003 - accuracy: 0.0362
Epoch 7/10
1050/1050 [==============================] - 0s 105us/sample - loss: 3.4002 - accuracy: 0.0333
Epoch 8/10
1050/1050 [==============================] - 0s 87us

1050/1050 [==============================] - 0s 71us/sample - loss: 3.4947 - accuracy: 0.0352
Epoch 6/10
1050/1050 [==============================] - 0s 72us/sample - loss: 3.4719 - accuracy: 0.0429
Epoch 7/10
1050/1050 [==============================] - 0s 72us/sample - loss: 3.4543 - accuracy: 0.0448
Epoch 8/10
1050/1050 [==============================] - 0s 70us/sample - loss: 3.4342 - accuracy: 0.0419
Epoch 9/10
1050/1050 [==============================] - 0s 70us/sample - loss: 3.4279 - accuracy: 0.0457
Epoch 10/10
1050/1050 [==============================] - 0s 111us/sample - loss: 3.3877 - accuracy: 0.0562
--- Starting trial: run-65
{'num_units': 256, 'dropout': 0.1, 'optimizer': 'Adagrad'}
Train on 1050 samples
Epoch 1/10
1050/1050 [==============================] - 0s 245us/sample - loss: 3.4140 - accuracy: 0.1019
Epoch 2/10
1050/1050 [==============================] - 0s 67us/sample - loss: 2.8797 - accuracy: 0.2505
Epoch 3/10
1050/1050 [==============================] - 0s 6

1050/1050 [==============================] - 0s 115us/sample - loss: 0.3912 - accuracy: 0.9162
--- Starting trial: run-71
{'num_units': 256, 'dropout': 0.1, 'optimizer': 'SGD'}
Train on 1050 samples
Epoch 1/10
1050/1050 [==============================] - 0s 239us/sample - loss: 3.3761 - accuracy: 0.0486
Epoch 2/10
1050/1050 [==============================] - 0s 63us/sample - loss: 3.1674 - accuracy: 0.1305
Epoch 3/10
1050/1050 [==============================] - 0s 65us/sample - loss: 2.9476 - accuracy: 0.2390
Epoch 4/10
1050/1050 [==============================] - 0s 62us/sample - loss: 2.7566 - accuracy: 0.3410
Epoch 5/10
1050/1050 [==============================] - 0s 64us/sample - loss: 2.5686 - accuracy: 0.4162
Epoch 6/10
1050/1050 [==============================] - 0s 61us/sample - loss: 2.4037 - accuracy: 0.4829
Epoch 7/10
1050/1050 [==============================] - 0s 62us/sample - loss: 2.2226 - accuracy: 0.5705
Epoch 8/10
1050/1050 [==============================] - 0s 64us/s

1050/1050 [==============================] - 0s 101us/sample - loss: 3.4196 - accuracy: 0.0371
Epoch 6/10
1050/1050 [==============================] - 0s 100us/sample - loss: 3.4316 - accuracy: 0.0381
Epoch 7/10
1050/1050 [==============================] - 0s 110us/sample - loss: 3.4001 - accuracy: 0.0324
Epoch 8/10
1050/1050 [==============================] - 0s 108us/sample - loss: 3.3998 - accuracy: 0.0381
Epoch 9/10
1050/1050 [==============================] - 0s 101us/sample - loss: 3.3999 - accuracy: 0.0352
Epoch 10/10
1050/1050 [==============================] - 0s 143us/sample - loss: 3.3992 - accuracy: 0.0381
--- Starting trial: run-78
{'num_units': 256, 'dropout': 0.9, 'optimizer': 'RMSprop'}
Train on 1050 samples
Epoch 1/10
1050/1050 [==============================] - 0s 340us/sample - loss: 5.4801 - accuracy: 0.0333
Epoch 2/10
1050/1050 [==============================] - 0s 80us/sample - loss: 3.4530 - accuracy: 0.0295
Epoch 3/10
1050/1050 [==============================] -

In [11]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 4296), started 0:05:45 ago. (Use '!kill 4296' to kill it.)